In [ ]:
# ==============================================================================
# ライブラリのインポート
# ==============================================================================
import torch
import torch.nn.functional as F
from diffusers import StableDiffusionPipeline, DDPMScheduler
from collections import Counter, defaultdict
import numpy as np
import json
from PIL import Image
from torchvision import datasets, transforms
from tqdm.auto import tqdm 
import os
import gc
import random
import pickle 

# ==============================================================================
# 1. パラメータとモデルの準備
# ==============================================================================

# --- モデル設定 ---
# 使用する拡散モデルのリポジトリID
STABLE_DIFFUSION_MODEL_ID = "runwayml/stable-diffusion-v1-5"

# --- データセット設定 ---
# stl10_binaryが保存されているルートディレクトリ 
DATA_ROOT = "./dataset"

# --- 実験パラメータ ---
# 1画像あたりに評価するサンプリング回数
SAMPLES_PER_IMAGE = 10
# ノイズを加える際のタイムステップの最小・最大値
NOISE_LEVEL_MIN = 100
NOISE_LEVEL_MAX = 800
# 実験に使用する画像の最大枚数 (Noneなら全画像)
NUM_IMAGES_TO_TEST = 10 # デバッグ用に減らす
# 乱数シード (再現性のため)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# --- 計算デバイスと精度 ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32

print(f"Using device: {DEVICE}")
print(f"Loading Standard Stable Diffusion pipeline: {STABLE_DIFFUSION_MODEL_ID}")

# --- モデルのロード ---
# Stable Diffusion パイプライン
pipe = StableDiffusionPipeline.from_pretrained(STABLE_DIFFUSION_MODEL_ID, torch_dtype=DTYPE).to(DEVICE)
vae = pipe.vae
tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder
unet = pipe.unet
scheduler = pipe.scheduler
pipe.set_progress_bar_config(disable=True)

# ==============================================================================
# 2. データセットのロード
# ==============================================================================

print(f"Loading STL-10 dataset (test split) from {DATA_ROOT}...")
try:
    # PIL画像を直接取得するため transform=None
    stl10_dataset_pil = datasets.STL10(root=DATA_ROOT, split='test', download=True) # download=True (初回実行時)
    class_names = stl10_dataset_pil.classes
    NUM_CLASSES = len(class_names)
except Exception as e:
    print(f"Error: Failed to load or download STL-10 dataset. Check the path: {DATA_ROOT}. Details: {e}")
    assert False

# --- 実験対象サブセットの作成 ---
all_indices = list(range(len(stl10_dataset_pil)))
if NUM_IMAGES_TO_TEST and NUM_IMAGES_TO_TEST < len(stl10_dataset_pil):
    print(f"Sampling {NUM_IMAGES_TO_TEST} indices randomly for this experiment.")
    # random.sampleでインデックスをランダムに選ぶ
    subset_indices = random.sample(all_indices, NUM_IMAGES_TO_TEST)
else:
    print("Using the full test dataset for this experiment.")
    subset_indices = all_indices
print(f"Running experiments on a subset of {len(subset_indices)} images.")

# ==============================================================================
# 3. ユーティリティ関数と事前計算
# ==============================================================================

# --- プロンプト生成関数 ---
def create_prompt(class_idx):
    """クラスインデックスからプロンプト文字列を生成"""
    name = class_names[class_idx]
    return f"a photo of a {name}"

# --- テキスト埋め込みの事前計算 ---
print("Pre-computing text embeddings for all classes...")
with torch.no_grad():
    uncond_input = tokenizer("", padding="max_length", max_length=tokenizer.model_max_length, return_tensors="pt")
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(DEVICE))[0]
    all_class_prompts = [create_prompt(i) for i in range(NUM_CLASSES)]
    all_text_embeddings_list = []
    batch_size = 64
    for i in tqdm(range(0, len(all_class_prompts), batch_size), desc="Encoding prompts"):
        prompts_batch = all_class_prompts[i:i+batch_size]
        text_inputs = tokenizer(prompts_batch, padding="max_length", max_length=tokenizer.model_max_length, return_tensors="pt", truncation=True)
        batch_embeddings = text_encoder(text_inputs.input_ids.to(DEVICE))[0]
        all_text_embeddings_list.append(batch_embeddings.cpu())
    all_text_embeddings = torch.cat(all_text_embeddings_list, dim=0).to(DEVICE) # 全クラスの埋め込みをGPUに保持
    print("Text embeddings pre-computation complete.")


# --- ノイズ予測誤差計算関数 (バッチ処理対応版) ---
@torch.no_grad()
def get_sd_noise_error_batched(init_latents, original_noise, text_embeds_batch, start_timestep):
    """指定されたタイムステップにおけるノイズ予測誤差をバッチ計算"""
    noisy_latents = scheduler.add_noise(init_latents, original_noise, torch.tensor([start_timestep], device=DEVICE))
    batch_size = text_embeds_batch.shape[0]
    latent_model_input = noisy_latents.repeat(batch_size, 1, 1, 1)
    timestep_input = torch.tensor([start_timestep], device=DEVICE).repeat(batch_size)
    pred_noise_batch = unet(latent_model_input, timestep_input, encoder_hidden_states=text_embeds_batch).sample
    original_noise_batch = original_noise.repeat(batch_size, 1, 1, 1)
    mse_losses = F.mse_loss(pred_noise_batch, original_noise_batch, reduction='none').mean(dim=(1, 2, 3))
    return mse_losses.cpu().numpy().tolist()

# ==============================================================================
# 4. メイン実行ループ
# ==============================================================================
all_results = [] # 全画像の結果をメモリ上に格納するリスト
sd_transform = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 実験対象の画像インデックスに対してループ
for image_index in tqdm(subset_indices, desc=f"Classifying {len(subset_indices)} images with Standard SD 1.5"):
    # --- 画像のロードと前処理 ---
    image_pil, true_label_idx = stl10_dataset_pil[image_index]
    if image_pil.mode != 'RGB': image_pil = image_pil.convert('RGB')
    image_tensor = sd_transform(image_pil).unsqueeze(0).to(DEVICE, dtype=DTYPE)

    # --- VAEエンコード ---
    with torch.no_grad():
        init_latents = vae.encode(image_tensor).latent_dist.sample() * vae.config.scaling_factor

    # --- 評価対象クラス ---
    candidate_indices = list(range(NUM_CLASSES)) # 0から9までのインデックス

    # --- テキスト埋め込みの準備 ---
    candidate_embeds = all_text_embeddings
    text_embeds_batch = torch.cat([uncond_embeddings, candidate_embeds])

    # --- サンプリングループ ---
    samples_data = []
    for _ in range(SAMPLES_PER_IMAGE):
        start_timestep = random.randint(NOISE_LEVEL_MIN, NOISE_LEVEL_MAX)
        noise = torch.randn_like(init_latents)
        # バッチ計算 (11プロンプト分)
        all_errors = get_sd_noise_error_batched(init_latents, noise, text_embeds_batch, start_timestep)
        sample_errors_mse = {'unconditional': all_errors[0]}
        # 2番目以降が各クラス(0~9)の誤差に対応
        for i, class_idx in enumerate(candidate_indices):
            sample_errors_mse[class_idx] = all_errors[i+1]
        samples_data.append({
            'used_timestep': start_timestep,
            'errors_mse': sample_errors_mse
        })

    # --- 画像ごとの結果を保存 ---
    all_results.append({
        'image_index': image_index,
        'true_label': true_label_idx,
        'candidates': candidate_indices, # 0~9のリスト
        'samples': samples_data
    })

    # メモリ解放
    del text_embeds_batch, init_latents, image_tensor
    gc.collect()
    torch.cuda.empty_cache()

print("\n--- Classification loop finished. Starting analysis... ---")

# ==============================================================================
# 5. 結果の分析と表示
# ==============================================================================

# --- 拡散モデルによる分類精度の集計 ---
print(f"Analyzing results using noise levels between {NOISE_LEVEL_MIN} and {NOISE_LEVEL_MAX} for majority vote.")
correct_count_min_error = 0 # Top-1の正解数のみカウント

for res in tqdm(all_results, desc="Analyzing results"):
    true_label = res['true_label']
    min_error_preds = []

    for sample in res['samples']:
        timestep = sample['used_timestep']
        # 全てのタイムステップのサンプルを多数決に使用 (NOISE_LEVEL_MIN ~ MAX)
        errors = sample['errors_mse']
        candidate_errors = {k: v for k, v in errors.items() if k != 'unconditional'}

        # 手法1: 最小誤差
        if candidate_errors:
            pred_min_error = min(candidate_errors, key=candidate_errors.get)
            min_error_preds.append(pred_min_error)

    # --- 多数決によるTop-1予測 ---
    if min_error_preds: # 投票がある場合のみ
        # 最も多く投票されたクラスを取得
        majority_vote_pred = Counter(min_error_preds).most_common(1)[0][0]
        # 正解と比較してカウント
        if majority_vote_pred == true_label:
            correct_count_min_error += 1

# --- 全体精度の表示 ---
total_images = len(all_results)
top1_accuracy = (correct_count_min_error / total_images) * 100 if total_images > 0 else 0

print(f"\n--- Accuracy for Majority Vote (Min Error) ---")
print(f"  -> Top-1 Accuracy: {top1_accuracy:.2f}% ({correct_count_min_error}/{total_images})")


# ==============================================================================
# 6. リソースの解放
# ==============================================================================
print("\nReleasing resources...")
del pipe, vae, tokenizer, text_encoder, unet, scheduler, all_text_embeddings, stl10_dataset_pil, all_results
gc.collect()
torch.cuda.empty_cache()

print(f"\n--- Standard SD 1.5 analysis finished for STL-10 (Top-1 Min Error only). ---")

Using device: cuda
Loading Standard Stable Diffusion pipeline: runwayml/stable-diffusion-v1-5


Loading pipeline components...: 100%|██████████| 7/7 [00:08<00:00,  1.23s/it]


Loading STL-10 dataset (test split) from C:/Users/suzuryo/Desktop/env/dataset...
Files already downloaded and verified
Sampling 10 indices randomly for this experiment.
Running experiments on a subset of 10 images.
Pre-computing text embeddings for all classes...


Encoding prompts: 100%|██████████| 1/1 [00:00<00:00,  4.93it/s]


Text embeddings pre-computation complete.


Classifying 10 images with Standard SD 1.5: 100%|██████████| 10/10 [05:50<00:00, 35.05s/it]



--- Classification loop finished. Starting analysis... ---
Analyzing results using noise levels between 100 and 800 for majority vote.


Analyzing results: 100%|██████████| 10/10 [00:00<?, ?it/s]



--- Accuracy for Majority Vote (Min Error) ---
  -> Top-1 Accuracy: 90.00% (9/10)

Releasing resources...

--- Standard SD 1.5 analysis finished for STL-10 (Top-1 Min Error only). ---


In [ ]:
# ==============================================================================
# ライブラリのインポート
# ==============================================================================
import torch
import torch.nn.functional as F
# Inpaintingパイプラインを使用
from diffusers import StableDiffusionInpaintPipeline, DDPMScheduler
from collections import Counter, defaultdict
import numpy as np
import json
from PIL import Image, ImageDraw # マスク生成用に ImageDraw を追加
from torchvision import datasets, transforms
from tqdm.auto import tqdm # プログレスバー表示用
import os
import gc # ガベージコレクション（メモリ解放）用
import random
import pickle # 結果表示に必要

# ==============================================================================
# 1. パラメータとモデルの準備
# ==============================================================================

# --- モデル設定 ---
# 使用する拡散モデルのリポジトリID (Inpaintingモデルに変更)
INPAINT_MODEL_ID = "runwayml/stable-diffusion-inpainting"

# --- データセット設定 ---
# stl10_binaryが保存されているルートディレクトリ
DATA_ROOT = "./dataset"

# --- 実験パラメータ ---
# 1画像あたりに評価するサンプリング回数
SAMPLES_PER_IMAGE = 10
# ノイズを加える際のタイムステップの最小・最大値
NOISE_LEVEL_MIN = 100
NOISE_LEVEL_MAX = 800
# 実験に使用する画像の最大枚数 (Noneなら全画像)
NUM_IMAGES_TO_TEST = 10 # デバッグ用に減らす
# 乱数シード (再現性のため)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# --- 計算デバイスと精度 ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32

print(f"Using device: {DEVICE}")
print(f"Loading Stable Diffusion Inpainting pipeline: {INPAINT_MODEL_ID}") # モデル名を変更

# --- モデルのロード ---
# Stable Diffusion Inpainting パイプライン
pipe = StableDiffusionInpaintPipeline.from_pretrained(INPAINT_MODEL_ID, torch_dtype=DTYPE).to(DEVICE) # パイプラインクラスを変更
vae = pipe.vae
tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder
unet = pipe.unet
scheduler = pipe.scheduler
pipe.set_progress_bar_config(disable=True)

# ==============================================================================
# 2. データセットのロード
# ==============================================================================

print(f"Loading STL-10 dataset (test split) from {DATA_ROOT}...")
try:
    stl10_dataset_pil = datasets.STL10(root=DATA_ROOT, split='test', download=True)
    class_names = stl10_dataset_pil.classes
    NUM_CLASSES = len(class_names)
except Exception as e:
    print(f"Error: Failed to load or download STL-10 dataset. Check the path: {DATA_ROOT}. Details: {e}")
    assert False

all_indices = list(range(len(stl10_dataset_pil)))
if NUM_IMAGES_TO_TEST and NUM_IMAGES_TO_TEST < len(stl10_dataset_pil):
    print(f"Sampling {NUM_IMAGES_TO_TEST} indices randomly for this experiment.")
    subset_indices = random.sample(all_indices, NUM_IMAGES_TO_TEST)
else:
    print("Using the full test dataset for this experiment.")
    subset_indices = all_indices
print(f"Running experiments on a subset of {len(subset_indices)} images.")

# ==============================================================================
# 3. ユーティリティ関数と事前計算
# ==============================================================================

# --- プロンプト生成関数 ---
def create_prompt(class_idx):
    """クラスインデックスからプロンプト文字列を生成"""
    name = class_names[class_idx]
    return f"a photo of a {name}"

# --- テキスト埋め込みの事前計算 ---
print("Pre-computing text embeddings for all classes...")
with torch.no_grad():
    uncond_input = tokenizer("", padding="max_length", max_length=tokenizer.model_max_length, return_tensors="pt")
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(DEVICE))[0]
    all_class_prompts = [create_prompt(i) for i in range(NUM_CLASSES)]
    all_text_embeddings_list = []
    batch_size = 64
    for i in tqdm(range(0, len(all_class_prompts), batch_size), desc="Encoding prompts"):
        prompts_batch = all_class_prompts[i:i+batch_size]
        text_inputs = tokenizer(prompts_batch, padding="max_length", max_length=tokenizer.model_max_length, return_tensors="pt", truncation=True)
        batch_embeddings = text_encoder(text_inputs.input_ids.to(DEVICE))[0]
        all_text_embeddings_list.append(batch_embeddings.cpu())
    all_text_embeddings = torch.cat(all_text_embeddings_list, dim=0).to(DEVICE)
    print("Text embeddings pre-computation complete.")


# --- ノイズ予測誤差計算関数 (Inpaintingモデル用 バッチ処理対応版) ---
@torch.no_grad()
def get_inpaint_noise_error_batched(
    init_latents,       # 元画像の潜在表現 (1, 4, H/8, W/8)
    original_noise,     # 加えるノイズ (1, 4, H/8, W/8)
    text_embeds_batch,  # テキスト埋め込み (N+1, seq_len, embed_dim)
    start_timestep,     # タイムステップ (int)
    mask_tensor         # マスク画像テンソル (1, 1, H, W) float16
):
    """Inpaintingモデルでノイズ予測誤差をマスク領域内で計算 (バッチ処理)"""
    # 1. ノイズ付加
    noisy_latents = scheduler.add_noise(init_latents, original_noise, torch.tensor([start_timestep], device=DEVICE))

    # 2. マスクとマスク済み画像を潜在空間サイズにリサイズ
    mask_latents = F.interpolate(mask_tensor, size=init_latents.shape[-2:]) # (1, 1, H/8, W/8)
    # 元画像に反転マスクを掛けてマスク部分を0にする
    masked_image_latents = init_latents * (1 - mask_latents) # (1, 4, H/8, W/8)

    # 3. U-Netへの入力 (9チャンネル) をバッチ処理用に準備
    batch_size = text_embeds_batch.shape[0]
    # 各入力をバッチサイズ分複製
    latent_model_input = noisy_latents.repeat(batch_size, 1, 1, 1) # (N+1, 4, ...)
    mask_input = mask_latents.repeat(batch_size, 1, 1, 1)         # (N+1, 1, ...)
    masked_input = masked_image_latents.repeat(batch_size, 1, 1, 1) # (N+1, 4, ...)
    # チャンネル次元(dim=1)で結合 -> (N+1, 9, H/8, W/8)
    final_input = torch.cat([latent_model_input, mask_input, masked_input], dim=1)
    # タイムステップもバッチサイズ分用意
    timestep_input = torch.tensor([start_timestep], device=DEVICE).repeat(batch_size)

    # 4. U-Netでノイズ予測を実行
    pred_noise_batch = unet(
        final_input,
        timestep_input,
        encoder_hidden_states=text_embeds_batch
    ).sample

    # 5. 誤差(MSE)をマスク領域内のみで計算
    # 元のノイズにマスクを掛ける
    original_noise_masked = original_noise * mask_latents # (1, 4, H/8, W/8)
    # バッチ内の各予測結果にもマスクを掛ける
    pred_noise_batch_masked = pred_noise_batch * mask_latents # (N+1, 4, H/8, W/8)
    # マスクされたノイズ同士でMSEを計算 (分母をマスク面積にするため工夫が必要だが、比較目的なら単純MSEでもOK)
    # ここでは単純なMSEを計算 (F.mse_lossはゼロ除算を避けるため、マスク外も含む全ピクセルで計算されるが、マスク外は0*0になるので結果的にマスク内誤差に近い値になる)
    mse_losses = F.mse_loss(pred_noise_batch_masked, original_noise_masked.repeat(batch_size, 1, 1, 1), reduction='none').mean(dim=(1, 2, 3))

    # 6. 結果をPythonのリストとして返す
    return mse_losses.cpu().numpy().tolist()

# ==============================================================================
# 4. メイン実行ループ
# ==============================================================================
all_results = []
sd_transform = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
# マスク画像をテンソルに変換するためのTransform
mask_transform = transforms.ToTensor()

# 実験対象の画像インデックスに対してループ
for image_index in tqdm(subset_indices, desc=f"Classifying {len(subset_indices)} images with Inpaint SD 1.5"):
    # --- 画像のロードと前処理 ---
    image_pil, true_label_idx = stl10_dataset_pil[image_index]
    if image_pil.mode != 'RGB': image_pil = image_pil.convert('RGB')
    image_tensor = sd_transform(image_pil).unsqueeze(0).to(DEVICE, dtype=DTYPE)

    # --- VAEエンコード ---
    with torch.no_grad():
        init_latents = vae.encode(image_tensor).latent_dist.sample() * vae.config.scaling_factor

    # --- 評価対象クラス ---
    candidate_indices = list(range(NUM_CLASSES))

    # --- テキスト埋め込みの準備 ---
    candidate_embeds = all_text_embeddings
    text_embeds_batch = torch.cat([uncond_embeddings, candidate_embeds])
    # --- サンプリングループ ---
    samples_data = []
    for _ in range(SAMPLES_PER_IMAGE):
        # 1. ランダムなタイムステップを選択
        start_timestep = random.randint(NOISE_LEVEL_MIN, NOISE_LEVEL_MAX)
        # 2. ランダムなノイズを生成
        noise = torch.randn_like(init_latents)
        # 3. ランダムな部分マスクを生成
        mask_pil = Image.new("L", (512, 512), 0) # 黒背景のマスク画像
        draw = ImageDraw.Draw(mask_pil)
        # 目標面積率 (10% ~ 80%)
        target_area_ratio = random.uniform(0.1, 0.8)
        img_area = 512 * 512
        mask_area = img_area * target_area_ratio
        # ランダムなアスペクト比 (例: 0.3 ~ 3.0)
        aspect_ratio = random.uniform(0.3, 3.0)
        # 幅と高さを計算
        mask_w = int(np.sqrt(mask_area / aspect_ratio))
        mask_h = int(mask_w * aspect_ratio)
        # 画像サイズ内に収める
        mask_w = max(16, min(mask_w, 512))
        mask_h = max(16, min(mask_h, 512))
        # ランダムな位置
        mask_x = random.randint(0, 512 - mask_w)
        mask_y = random.randint(0, 512 - mask_h)
        # マスクを描画 (白=255 で塗りつぶし)
        draw.rectangle((mask_x, mask_y, mask_x + mask_w, mask_y + mask_h), fill=255)
        # マスクをテンソルに変換
        mask_tensor = mask_transform(mask_pil).unsqueeze(0).to(DEVICE, dtype=DTYPE) # (1, 1, 512, 512)

        # 4. ノイズ予測誤差をバッチ計算 (Inpainting関数を使用)
        all_errors = get_inpaint_noise_error_batched(
            init_latents, noise, text_embeds_batch, start_timestep, mask_tensor
        )

        # 5. 結果を整理
        sample_errors_mse = {'unconditional': all_errors[0]}
        for i, class_idx in enumerate(candidate_indices):
            sample_errors_mse[class_idx] = all_errors[i+1]
        samples_data.append({
            'used_timestep': start_timestep,
            'errors_mse': sample_errors_mse
        })

    # --- 画像ごとの結果を保存 ---
    all_results.append({
        'image_index': image_index,
        'true_label': true_label_idx,
        'candidates': candidate_indices,
        'samples': samples_data
    })

    # メモリ解放
    del text_embeds_batch, init_latents, image_tensor, mask_tensor
    gc.collect()
    torch.cuda.empty_cache()

print("\n--- Classification loop finished. Starting analysis... ---")

# ==============================================================================
# 5. 結果の分析と表示
# ==============================================================================
print(f"Analyzing results using noise levels between {NOISE_LEVEL_MIN} and {NOISE_LEVEL_MAX} for majority vote.")
correct_count_min_error = 0

for res in tqdm(all_results, desc="Analyzing results"):
    true_label = res['true_label']
    min_error_preds = []
    for sample in res['samples']:
        errors = sample['errors_mse']
        candidate_errors = {k: v for k, v in errors.items() if k != 'unconditional'}
        if candidate_errors:
            pred_min_error = min(candidate_errors, key=candidate_errors.get)
            min_error_preds.append(pred_min_error)

    if min_error_preds:
        majority_vote_pred = Counter(min_error_preds).most_common(1)[0][0]
        if majority_vote_pred == true_label:
            correct_count_min_error += 1

total_images = len(all_results)
top1_accuracy = (correct_count_min_error / total_images) * 100 if total_images > 0 else 0

print(f"\n--- Accuracy for Majority Vote (Min Error) ---")
print(f"  -> Top-1 Accuracy: {top1_accuracy:.2f}% ({correct_count_min_error}/{total_images})")

# ==============================================================================
# 6. リソースの解放
# ==============================================================================
print("\nReleasing resources...")
del pipe, vae, tokenizer, text_encoder, unet, scheduler, all_text_embeddings, stl10_dataset_pil, all_results
gc.collect()
torch.cuda.empty_cache()

print(f"\n--- Inpaint SD 1.5 analysis finished for STL-10 (Top-1 Min Error only). ---") # メッセージ変更

c:\Users\suzuryo\.conda\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Loading Stable Diffusion Inpainting pipeline: runwayml/stable-diffusion-inpainting


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\suzuryo\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\suzuryo\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...:  29%|██▊       | 2/7 [00:05<00:12,  2.52s/it]An error occurred while trying to fetch C:\Users\suzuryo\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a76071f7280aedbdb3253bdb9e9d5d84bb\vae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\suzuryo\.cache\huggingface\hub\models--runwayml--stable-diffusion-inpainting\snapshots\8a4288a7607

Loading STL-10 dataset (test split) from C:/Users/suzuryo/Desktop/env/dataset...
Files already downloaded and verified
Sampling 10 indices randomly for this experiment.
Running experiments on a subset of 10 images.
Pre-computing text embeddings for all classes...


Encoding prompts: 100%|██████████| 1/1 [00:00<00:00, 10.11it/s]


Text embeddings pre-computation complete.


Classifying 10 images with Inpaint SD 1.5: 100%|██████████| 10/10 [00:35<00:00,  3.57s/it]



--- Classification loop finished. Starting analysis... ---
Analyzing results using noise levels between 100 and 800 for majority vote.


Analyzing results: 100%|██████████| 10/10 [00:00<?, ?it/s]


--- Accuracy for Majority Vote (Min Error) ---
  -> Top-1 Accuracy: 60.00% (6/10)

Releasing resources...

--- Inpaint SD 1.5 analysis finished for STL-10 (Top-1 Min Error only). ---
